In [4]:
# Источники вдохновения #
# BackProp #
    # https://neurohive.io/ru/osnovy-data-science/obratnoe-rasprostranenie/
# Нейроны, ф-ии активации и суммирования #
    # https://stepik.org/lesson/24679/step/4?unit=7041
# Примеры #
    # https://neurohive.io/ru/tutorial/kak-sozdat-nejronnuju-set-s-nulja-na-jazyke-python/
    # https://python-scripts.com/intro-to-neural-networks
    # https://neuralnet.info/chapter/персептроны/#Персептрон-с-одним-скрытым-слоем
# Код из 1-го примера #
    # https://gist.github.com/jamesloyys/ff7a7bb1540384f709856f9cdcdee70d#file-neural_network_backprop-py
# Нормализация данных #
    # http://neuropro.ru/mypapers/neurcompmag03_1.pdf
    # https://ru.stackoverflow.com/questions/523343/Вопрос-по-архитектуре-нейронной-сетимногослойного-перцептрона/570504#570504

In [54]:
# библиотеки и входные данные #
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# РАБОТАЕТ!!! #

# тест #
# на выходе 0, если 0 > 1 #
# на выходе 1, если 1 > 0 #
x = np.array(([0,0,1],[0,1,1],[1,0,1],[1,1,0]), dtype=float)
y = np.array(([0],[1],[1],[1]), dtype=float)

# нормальные входные данные #
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
X = dataset.data
Y = dataset.target

In [55]:
Y.shape  = (Y.shape[0], 1)

In [56]:
# Разделение набора данных #
test_size = 0.2
seed = 42

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Попытка в нормализацию данных #

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


# Новая попытка #

#mean = X_train.mean(axis=0)
#std  = X_train.std(axis=0)

#X_train -= mean
#X_train /= std
#X_test  -= mean
#X_test  /= std


In [57]:
X_train = X_train[:,0:29]
X_test  = X_test[:,0:29]
Y_train.shape = (Y_train.shape[0], 1)
Y_test.shape  = (Y_test.shape[0], 1)
X_train.shape

(455, 29)

In [58]:
# Функция активации #
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [59]:
# Производная ф-ии активации #
def sigmoid_d(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

In [62]:
# Класс нейронки #
class NeuralNetwork:
    def __init__(self, x, y, n1_count, n2_count):
        # Входные данные #
        self. y     = y
        self.input  = x
        self.output = np.zeros(len(y))
        
        # Веса #
        # 1 - кол-во входов, 2 - кол-во нейронов #
         
                   
        # Веса первого слоя #
        self.w1 = np.random.randn(self.input.shape[1], n1_count) / np.sqrt(n1_count)
        # Веса второго слоя #
        self.w2 = np.random.randn(n1_count, n2_count) / np.sqrt(n2_count)
        
        # Веса выходного слоя #
        self.w_out = np.random.randn(n2_count, 1)
        
    def FeedForward(self, x, y):
        self.k1    = sigmoid(np.dot(x, self.w1))
        self.k2    = sigmoid(np.dot(self.k1, self.w2))
        
        #print('k1 - ' + str(self.k1))
        #print('k2 - ' + str(self.k2))

        self.k_out = sigmoid(np.dot(self.k2, self.w_out))
        
        return self.k_out
        
    def BackProp(self, x, y):
        # Упрощения #
        d_pred = 2 * (y - self.k_out) # Это не точно! #
        d_out  = d_pred*sigmoid_d(self.output) # Для выходного слоя #
        
        # Тут могут быть ошибки # # Но их нет #
        d_w_out = np.dot(self.k2.T, d_out)
        d_w2    = np.dot(self.k1.T, np.dot(d_out, self.w_out.T)*sigmoid_d(self.k2))
        d_w1    = np.dot(x.T, np.dot(np.dot(d_out, self.w_out.T)*sigmoid_d(self.k2), self.w2.T)*sigmoid_d(self.k1))
        
        # Место для тестов #
        # Место для тестов #
        
        # Обновление весов #
        self.w1    += d_w1
        self.w2    += d_w2
        self.w_out += d_w_out
        
    def train(self, x, y):
        self.output = self.FeedForward(x, y)
        self.BackProp(x, y)

In [67]:
# Кол-во нейронов в скрытых слоях #
n1_count = int(input("Укажите кол-во нейронов в 1-м слое: "))
n2_count = int(input("Укажите кол-во нейронов во 2-м слое: "))

# Ограничение, ну так, просто #
if n1_count <= 0:
    n1_count = 1
    
if n2_count <= 0:
    n2_count = 1

# Объявление класса #
nn = NeuralNetwork(X_train_scaled, Y_train, n1_count, n2_count)

# Тренировка #
for i in range(5000):
    if i % 2500 == 0:
        print ("for iteration # " + str(i) + "\n")
        #print ("Input : \n" + str(X_train_scaled))
        print ("Actual Output: \n" + str(Y_train))
        print ("Predicted Output: \n" + str(nn.output))
        print ("Loss: \n" + str(np.mean(np.square(Y_train - nn.output)))) # mean sum squared loss
        print ("\n")
    nn.train(X_train_scaled, Y_train)

Укажите кол-во нейронов в 1-м слое: 128
Укажите кол-во нейронов во 2-м слое: 64
for iteration # 0

Actual Output: 
[[1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]


c:\users\godoftea\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


for iteration # 2500

Actual Output: 
[[1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [

In [70]:
def predict(x, y):
    
    n = x.shape[0]
    p = np.zeros_like(y)
    
    test = nn.FeedForward(x, y)
    
    for i, j in enumerate(test):
        if j > 0.5:
            p[i] = 1
    
    print("Accuracy: "  + str(np.sum((p == y)/n)))
    
predict(X_test_scaled, Y_test)

Accuracy: 0.9912280701754386


c:\users\godoftea\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
